In [26]:
import pandas as pd
import json

In [27]:
!ls /home/leonhards/projects/tuplex-public/benchmarks/nextconf/hyperspecialization/github/local-exp/c++-baseline/github

best		  cjson		     cstruct		  yyjson
best_results.csv  cjson_results.csv  cstruct_results.csv  yyjson_results.csv


In [28]:
df = pd.DataFrame()

In [29]:
import glob

In [30]:
rpath = '/home/leonhards/projects/tuplex-public/benchmarks/nextconf/hyperspecialization/github/local-exp/c++-baseline/github'

In [31]:
for path in glob.glob(rpath + '/*.csv'):
    df = pd.concat((df, pd.read_csv(path)))

In [32]:
df.groupby('mode').mean(numeric_only=True)

""
mode
best
cjson
cstruct
mode
yyjson


In [33]:
df['total_time_in_s'] = pd.to_numeric(df['total_time_in_s'], errors='coerce')

In [34]:
df.dropna().groupby('mode').mean()

/tmp/ipykernel_86324/3359699390.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.dropna().groupby('mode').mean()


,total_time_in_s
mode,
best,22.072186
cjson,155.714987
cstruct,25.833117
yyjson,73.006668


In [35]:
def load_tuplex():
    paths = [
             '/home/leonhards/projects/tuplex-public/benchmarks/nextconf/hyperspecialization/github/local-exp/python-baseline/github/results.ndjson',
             '/home/leonhards/projects/tuplex-public/benchmarks/nextconf/hyperspecialization/github/local-exp/tuplex/github/hyper/results.ndjson',
             '/home/leonhards/projects/tuplex-public/benchmarks/nextconf/hyperspecialization/github/local-exp/tuplex/github/hyper-noopt/results.ndjson',
             '/home/leonhards/projects/tuplex-public/benchmarks/nextconf/hyperspecialization/github/local-exp/tuplex/github/nohyper/results.ndjson',
            ]
    def extract_per_file_stats_tuplex(job_stats):
        n_requests = len(job_stats['responses'])
        L =  [job_stats['responses'][i]['stats'] for i in range(n_requests)]
        for i in range(n_requests):
            req_uri = job_stats['responses'][i]['request'][0]
            L[i]['input_path'] = req_uri[:req_uri.rfind(':')]
            L[i]['req_uri'] = req_uri
            L[i]['duration'] = L[i]['request_total_time']
            L[i]['num_input_rows'] = L[i]['input']['total_input_row_count']
            L[i]['num_output_rows'] = L[i]['output']['normal']
        return L
    data = []
    
    for path in paths:
        print(f'>>> processing {path}')
        with open(path) as fp:
            lines = fp.readlines()
            rows = [json.loads(line) for line in lines]
    
            print(len(lines))
            # row = rows[-1]
            for row in rows:
                if 'python-baseline' not in path:
                    ans = {'benchmark': 'github'}
                    for k in ['benchmark', 'input_path', 'job_time_in_s', 'metrics', 'mode', 'options', 'output_path', 'scratch_path', 'startup_time_in_s']:
                        ans[k] = row.get(k)
                    try:
                        ans['per_file_stats'] = extract_per_file_stats_tuplex(row['detailed_job_stats'])
                    except:
                        print('--- ERR: extract failed')
                    row = ans
                data.append(row)
                
            del row
            del rows
            del lines
    df = pd.DataFrame(data)        
    return df

In [36]:
df_tplx= load_tuplex()

>>> processing /home/leonhards/projects/tuplex-public/benchmarks/nextconf/hyperspecialization/github/local-exp/python-baseline/github/results.ndjson
5
>>> processing /home/leonhards/projects/tuplex-public/benchmarks/nextconf/hyperspecialization/github/local-exp/tuplex/github/hyper/results.ndjson
4
>>> processing /home/leonhards/projects/tuplex-public/benchmarks/nextconf/hyperspecialization/github/local-exp/tuplex/github/hyper-noopt/results.ndjson
4
>>> processing /home/leonhards/projects/tuplex-public/benchmarks/nextconf/hyperspecialization/github/local-exp/tuplex/github/nohyper/results.ndjson
4


In [37]:
df_tplx.groupby('output_path').mean()

/tmp/ipykernel_86324/556165960.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_tplx.groupby('output_path').mean()


,job_time_in_s,startup_time_in_s,total_input_paths_size_in_bytes,total_input_rows,total_output_rows
output_path,,,,,
./local-exp/python-baseline/github/output,138.595526,0.000000,3.807602e+10,11012665.0,294195.0
./local-exp/tuplex/github/hyper-noopt/output,170.382859,0.020130,NaN,NaN,NaN
./local-exp/tuplex/github/hyper/output,171.051636,0.020238,NaN,NaN,NaN
./local-exp/tuplex/github/nohyper/output,182.090354,0.020099,NaN,NaN,NaN


In [38]:
df.dropna().groupby('mode').mean()

/tmp/ipykernel_86324/3359699390.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.dropna().groupby('mode').mean()


,total_time_in_s
mode,
best,22.072186
cjson,155.714987
cstruct,25.833117
yyjson,73.006668
